In [1]:
import pandas as pd
from datetime import datetime, timedelta
import re
from difflib import get_close_matches
import warnings
warnings.filterwarnings('ignore')

# Read the Excel files
countries_df = pd.read_excel("My port\My port\Countries IDs.xlsx")
codes_df = pd.read_excel("My port\My port\Ports_ceties_codes_and_IDs.xlsx")
my_port_aug_file = pd.ExcelFile("My port\My port\MyPoert AUG sheet1.xlsx")

# Define regex patterns for matching variations of 'Trip' and 'Price Offer'
trip_pattern = re.compile(r'Trip\s*', flags=re.IGNORECASE)
price_offer_pattern = re.compile(r'Price\s*Offer|Price\s*', flags=re.IGNORECASE)

# Function to find approximate matches
def find_approximate_match(word, choices):
    # Use difflib's get_close_matches to find approximate matches
    matches = get_close_matches(word, choices, n=1, cutoff=0.8)
    return matches[0] if matches else None

# Initialize an empty dataframe to store the combined results
combined_results = pd.DataFrame()

# Iterate over each sheet and process the data
for sheet_name in my_port_aug_file.sheet_names:
    my_port_aug_df = pd.read_excel(my_port_aug_file, sheet_name=sheet_name)
    my_port_aug_df = my_port_aug_df.iloc[:, :2] 
    my_port_aug_df = my_port_aug_df.dropna()

    # Find the columns containing trip and price offer information dynamically
    trip_column = [col for col in my_port_aug_df.columns if trip_pattern.search(col)]
    price_offer_column = [col for col in my_port_aug_df.columns if price_offer_pattern.search(col)]
    
    if len(trip_column) == 1:
        trip_column = trip_column[0]
    else:
        raise ValueError("Trip column not found or multiple Trip columns present")

    if len(price_offer_column) == 1:
        price_offer_column = price_offer_column[0]
    else:
        raise ValueError("Price Offer column not found or multiple Price Offer columns present")

    # Clean the Trip column by removing leading/trailing whitespace
    my_port_aug_df[trip_column] = my_port_aug_df[trip_column].str.strip()

    # Loop through each row in the dataframe
    for index, row in my_port_aug_df.iterrows():
        # Split the Trip string by "FROM" and "TO"
        trip_parts = row[trip_column].split("TO")
        if len(trip_parts) != 2:
            continue  # Skip rows with incorrect format

        # Find approximate match for the 'FROM' and 'TO' cities
        from_city = find_approximate_match(trip_parts[0].split("FROM")[-1].strip(), codes_df['Port Name'])
        to_city = find_approximate_match(trip_parts[1].strip(), codes_df['Port Name'])

        if from_city and to_city:
            # Retrieve the corresponding Code and ID for both cities
            from_code_id = codes_df.loc[codes_df['Port Name'] == from_city, ['Code', 'ID']].values
            to_code_id = codes_df.loc[codes_df['Port Name'] == to_city, ['Code', 'ID']].values

            # Update the Trip column with the corrected values
            my_port_aug_df.at[index, trip_column] = f"FROM {from_city} TO {to_city}"

            # Add Code and ID to the DataFrame
            my_port_aug_df.at[index, 'From Code'] = from_code_id[0][0]
            my_port_aug_df.at[index, 'From ID'] = from_code_id[0][1]
            my_port_aug_df.at[index, 'To Code'] = to_code_id[0][0]
            my_port_aug_df.at[index, 'To ID'] = to_code_id[0][1]
            
            # Extract trip_type from the Price Offer column
            if "ONE WAY" in row[price_offer_column].upper():
                my_port_aug_df.at[index, 'trip_type'] = 'O'
            elif "ROUND" in row[price_offer_column].upper():
                my_port_aug_df.at[index, 'trip_type'] = 'R'
                
            # Assign values to 'ID_to' and 'URL' only if both 'From_Port' and 'To_Port' are successfully matched
            my_port_aug_df['URL'] = my_port_aug_df['From Code'] + '-' + my_port_aug_df['To Code'] + '-' + my_port_aug_df['trip_type']
            my_port_aug_df['URL'] = my_port_aug_df['URL'].str.lower()    

            # Assign publish_date and expir_date
            my_port_aug_df.at[index, 'publish_date'] = datetime.now().strftime('%Y-%m-%d')
            publish_date = datetime.now()
            my_port_aug_df.at[index, 'expir_date'] = (publish_date + timedelta(days=35)).strftime('%Y-%m-%d')

        else:
            # Handle cases where approximate matches are not found
            print(f"Approximate match not found for row {index}")

    # Extract port names from Trip column
    my_port_aug_df['From_Port'] = my_port_aug_df[trip_column].str.extract(r'FROM\s+(.*?)\s+TO', flags=re.IGNORECASE)
    my_port_aug_df['To_Port'] = my_port_aug_df[trip_column].str.extract(r'TO\s+(.*?)\s+', flags=re.IGNORECASE)

    # Append the processed data to the combined results dataframe
    combined_results = combined_results.append(my_port_aug_df[['From ID', 'To ID', 'trip_type', 'URL', 'publish_date', 'expir_date']], ignore_index=True)

# Write the combined results to a single output Excel file
combined_results.to_excel("offers_tamplet.xlsx", index=False)


Approximate match not found for row 1
Approximate match not found for row 2
Approximate match not found for row 10
Approximate match not found for row 12


In [3]:
import pandas as pd
from datetime import datetime, timedelta
import re
from difflib import get_close_matches
import warnings
warnings.filterwarnings('ignore')

# Read the Excel files
countries_df = pd.read_excel("My port\My port\Countries IDs.xlsx")
codes_df = pd.read_excel("My port\My port\Ports_ceties_codes_and_IDs.xlsx")
my_port_aug_file = pd.ExcelFile("My port\My port\MyPoert AUG sheet1.xlsx")

# Define regex patterns for matching variations of 'Trip' and 'Price Offer'
trip_pattern = re.compile(r'Trip\s*', flags=re.IGNORECASE)
price_offer_pattern = re.compile(r'Price\s*Offer|Price\s*', flags=re.IGNORECASE)

# Function to find approximate matches
def find_approximate_match(word, choices):
    # Use difflib's get_close_matches to find approximate matches
    matches = get_close_matches(word, choices, n=1, cutoff=0.8)
    return matches[0] if matches else None

# Initialize an empty dataframe to store the combined results
combined_results = pd.DataFrame()

# Iterate over each sheet and process the data
for sheet_name in my_port_aug_file.sheet_names:
    my_port_aug_df = pd.read_excel(my_port_aug_file, sheet_name=sheet_name)
    my_port_aug_df = my_port_aug_df.iloc[:, :2] 
    my_port_aug_df = my_port_aug_df.dropna()

    # Initialize country_id column
    my_port_aug_df['country_id'] = None

    # Find the columns containing trip and price offer information dynamically
    trip_column = [col for col in my_port_aug_df.columns if trip_pattern.search(col)]
    price_offer_column = [col for col in my_port_aug_df.columns if price_offer_pattern.search(col)]
    
    if len(trip_column) == 1:
        trip_column = trip_column[0]
    else:
        raise ValueError("Trip column not found or multiple Trip columns present")

    if len(price_offer_column) == 1:
        price_offer_column = price_offer_column[0]
    else:
        raise ValueError("Price Offer column not found or multiple Price Offer columns present")

    # Clean the Trip column by removing leading/trailing whitespace
    my_port_aug_df[trip_column] = my_port_aug_df[trip_column].str.strip()

    # Loop through each row in the dataframe
    for index, row in my_port_aug_df.iterrows():
        # Split the Trip string by "FROM" and "TO"
        trip_parts = row[trip_column].split("TO")
        if len(trip_parts) != 2:
            continue  # Skip rows with incorrect format

        # Find approximate match for the 'FROM' and 'TO' cities
        from_city = find_approximate_match(trip_parts[0].split("FROM")[-1].strip(), codes_df['Port Name'])
        to_city = find_approximate_match(trip_parts[1].strip(), codes_df['Port Name'])

        if from_city and to_city:
            # Retrieve the corresponding Code and ID for both cities
            from_code_id = codes_df.loc[codes_df['Port Name'] == from_city, ['Code', 'ID']].values
            to_code_id = codes_df.loc[codes_df['Port Name'] == to_city, ['Code', 'ID']].values

            # Update the Trip column with the corrected values
            my_port_aug_df.at[index, trip_column] = f"FROM {from_city} TO {to_city}"

            # Add Code and ID to the DataFrame
            my_port_aug_df.at[index, 'From Code'] = from_code_id[0][0]
            my_port_aug_df.at[index, 'From ID'] = from_code_id[0][1]
            my_port_aug_df.at[index, 'To Code'] = to_code_id[0][0]
            my_port_aug_df.at[index, 'To ID'] = to_code_id[0][1]
            
            # Extract trip_type from the Price Offer column
            if "ONE WAY" in row[price_offer_column].upper():
                my_port_aug_df.at[index, 'trip_type'] = 'O'
            elif "ROUND" in row[price_offer_column].upper():
                my_port_aug_df.at[index, 'trip_type'] = 'R'
                
            # Assign values to 'ID_to' and 'URL' only if both 'From_Port' and 'To_Port' are successfully matched
            my_port_aug_df['URL'] = my_port_aug_df['From Code'] + '-' + my_port_aug_df['To Code'] + '-' + my_port_aug_df['trip_type']
            my_port_aug_df['URL'] = my_port_aug_df['URL'].str.lower()    

            # Assign publish_date and expir_date
            my_port_aug_df.at[index, 'publish_date'] = datetime.now().strftime('%Y-%m-%d')
            publish_date = datetime.now()
            my_port_aug_df.at[index, 'expir_date'] = (publish_date + timedelta(days=35)).strftime('%Y-%m-%d')
            
            # Extract country name from the sheet name
            country_name = sheet_name
            
            # Look up the country ID from the countries dataframe if there are matching rows
            country_id_series = countries_df.loc[countries_df['Country'] == country_name, 'ID']
            if not country_id_series.empty:
                country_id = country_id_series.iloc[0]

                # Add country ID to the DataFrame
                my_port_aug_df.at[index, 'country_id'] = country_id
            else:
                print(f"No matching country found for sheet '{sheet_name}'")

        else:
            # Handle cases where approximate matches are not found
            print(f"Approximate match not found for row {index}")

    # Append the processed data to the combined results dataframe
    combined_results = combined_results.append(my_port_aug_df[['URL', 'country_id']], ignore_index=True)

# Write the combined results to a single output Excel file
combined_results.to_excel("offer_content.xlsx", index=False)


Approximate match not found for row 1
No matching country found for sheet 'Paramaribo'
No matching country found for sheet 'Paramaribo'
No matching country found for sheet 'Paramaribo'
No matching country found for sheet 'Paramaribo'
No matching country found for sheet 'Paramaribo'
No matching country found for sheet 'Paramaribo'
No matching country found for sheet 'Paramaribo'
No matching country found for sheet 'Paramaribo'
No matching country found for sheet 'Paramaribo'
No matching country found for sheet 'Paramaribo'
No matching country found for sheet 'Paramaribo'
Approximate match not found for row 2
No matching country found for sheet 'Majuro'
No matching country found for sheet 'Majuro'
No matching country found for sheet 'Majuro'
No matching country found for sheet 'Majuro'
No matching country found for sheet 'Majuro'
No matching country found for sheet 'Majuro'
No matching country found for sheet 'Majuro'
No matching country found for sheet 'Majuro'
No matching country found